In [1]:
#Universidad Nacional Autónoma de México
#Posgrado en Ciencias de la Tierra 
#Campo 1, Geofísica de la Tierra sólida, Sismología

#Elaborado por Isaac Valverde, 2023

#Programa 6: Notebook de Python 3 que calcula los coeficientes de variación a partir de la estrategia semianalítica de Nocquet (2018) utilizando el estimador cuasi-Monte Carlo
#Basado en la estrategia semianalítica de Nocquet (2018)

#Nota: Las ecuaciones referidas en este notebook corresponden al articulo "Stochastic static fault slip inversion from geodetic data with non-negativity and bound constraints" Nocquet (2018)

In [2]:
from scipy import special
from scipy.stats.mvn import mvnun
from scipy.stats import qmc
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
#Función que intercambia las filas de una matriz o vector columna, colocando el o los reglones indicados comenzando en el primer renglón 
#Entradas
#A: Matriz o vector cuyas filas serán reacomodadas
#ind: Lista con el o los números de la fila que se van a colocar en el primer renglón  
#Salida
#A1: Matriz reordenada

def reorder(A,ind):
    #Se obtienen las dimensiones de la matriz o vector a reordenar
    n1,n2=np.shape(A)
    #Se define un arreglo auxiliar para introducir los renglones que se colocan en la posición del primer renglón
    aux1=np.zeros((len(ind),n2))
    #Se genera una copia de A
    aux2=np.copy(A)
    #Se inicia un ciclo para colocar el o los renglones de interés en aux1
    for i in range (0,len(ind)):
        aux1[i]=np.copy(A[ind[i]-1])
    #Se eliminan los renglones a reacomodar de la copia de A
    aux2=np.delete(A,ind-1,axis=0)
    #Se unen los renglónes de interes con el resto (en el orden que tienen omitiendo los renglones de interés)
    A1=np.concatenate((aux1, aux2), axis=0)
    return A1

In [4]:
#Funcion que calcula el vector de medias y la matriz de covarianza de la función de densidad de probabilidad posterior a partir de las expresiones de Tarantola y Valette (1982)
#Entradas
#G: Matriz kernel de la inversión (dimensión número de datos por número de parámetros) 
#d: Vector columna que contiene los datos de desplazamiento (de dimensión 1 por número de datos)
#Cdinv: Matriz de covarianza de los datos
#alpha:Valor de alpha para aproximar la distribución apriori
#limsup: Vector que contiene el límite superior del intervalo de truncamiento de cada parámetro
#m0: Media de la distribucion apriori
#Salida
#mv: Vector de medias de la posterior
#Cmv: Matriz de covarianza de la posterior

def mvCmv(G,d,Cdinv,alpha,limsup,m0):
    
    #Se obtiene el número de parámetros
    Nm=np.shape(G)[1]
    #Se obtiene la transpuesta del kernel G
    GT=np.transpose(G)
    #Se define la desviación estandar (pp 373, primer párrafo, quinto renglón)
    sigma=alpha*(limsup/2)
    #Se define la matriz de covarianza de los parámetros (pp) (pp 372, primer parrafo, segundo renglón)
    Cm=np.identity(Nm)*(((np.ones(Nm)*sigma).reshape(Nm,1))**2)
    #Se define la inversa la matriz de covarianza de los parámetros
    Cminv=np.linalg.inv(Cm)
    #Se define la matriz de covarianza posterior (Ecuaciones de Tarantola y Valette, pp. 369, Ec. 7)
    Cmv=np.linalg.inv(np.matmul(np.matmul(GT,Cdinv),G)+Cminv)
    #Se define la transpuesta de la matriz de covarianza posterior
    CmvT=np.transpose(Cmv)
    #Se define la inversa de la matriz de covarianza posterior
    Cmvinv=np.matmul(np.matmul(GT,Cdinv),G)+Cminv
    #Se define la expectativa (Ecuaciones de Tarantola y Valette, pp. 369, Ec. 6)
    mv=np.matmul(np.linalg.inv(np.matmul(np.matmul(GT,Cdinv),G)+Cminv), np.matmul(np.matmul(GT,Cdinv),d)+np.matmul(Cminv,m0))
    return mv, Cmv

In [5]:
#Función que estima la integral de una función exponencial gaussiana mediante el método de Monte Carlo
#Entradas
#bm1: Vector de medias de la función exponencial gaussiana
#Ainv: Matriz de covarianza de la función exponencial gaussiana
#RND:Conjunto de números aleatorios en los que se evalúa la función para estimar el valor de la integral
#V: Hipervolumen de la región de integración
#Salida
#I: Valor estimado de la integral

#Funcion que calcula las marginales de una distribución de probabilidad (caso 1 y 2D)
def ExpIntegral(bm1,Ainv,RND,V):
    #Se obtiene el número de parámetros
    NM=np.shape(Ainv)[0]
    #Se obtiene el número de muestras que se van a usar para evaluar la función
    NoMuestras=np.shape(RND)[0]
    #Se inicializa una variable auxiliar para acumular la suma de los valores de la funcion evaluada en las muestras
    Iaux=0
    #Se inicia un ciclo para evaluar las muestras en la función
    for i in range(0,int(NoMuestras)): 
        #Se extrae la i-esima muestra del arreglo que contiene todas las muestras
        m2=RND[i,:].reshape(NM,1)
        #Se evalua la función y se suma al resto 
        Iaux=Iaux+np.exp(-0.5*np.matmul(np.transpose(m2-bm1),np.matmul(Ainv,m2-bm1)))
    #Se multiplica la suma de las evaluaciones de la función en las muestras por el hipervolumen de la región de integración y se divide entre el número de muestras
    I=(Iaux*V)/NoMuestras
    return I

In [6]:
#Función que calcula la marginal de una TMVN utilizando el estimador Monte Carlo para resolver las integrales
#Entradas
#m: Matriz que contiene el espacio de parámetros descretizado  
#Cmv1: Matriz de covarianza de la TMVN posterior
#mv1: Vector (columna) de medias de la TMVN posterior
#Ind1: Número de parámetro del cual se obtiene la marginal
#RND: Conjunto de números aleatorios que se utiliza para estimar las integrales mediante Monte Carlo (las filas son el número de muestras y las columnas el valor de cada parámetro)
#V: Hipervolumen de la región de integración
#Salida
#sigmMb1: Vector que contiene los valores de la marginal

def marginal_MC(m,Cmv1,mv1,Ind1,RND,V): 
    #Se obtiene una lista con todas las posibles combinaciones del espacio de parámetros
    Ind=np.array(Ind1)
    
    #Se acomodan las filas y la columnas de la matriz de covarianza posterior de los parámetros en función de la marginal requerida
    Cmv0=np.transpose(reorder(np.transpose(Cmv1),Ind))
    Cmv=reorder(Cmv0,Ind)

    #Se acomodan los elementos del vector expectativa en función de la marginal requerida
    mv=reorder(mv1,Ind)
    
    #Se obtienen la dimensión de la submatriz m1
    NmM=len(Ind)
    #Se obtiene el número de parámetros
    Nm=np.shape(m)[1]
    #Se obtiene el vector recorrido de las marginales
    mn=m[0:len(m),0]
    
    #Se definen las submatrices necesarias para las operaciones del calculo de las marginales
    Cmvinv11=np.linalg.inv(np.array([Cmv[0:NmM,0:NmM]]).reshape(NmM,NmM))
    Cmv12=Cmv[0:NmM,NmM:Nm].reshape(NmM,Nm-NmM)
    CmvT12=np.transpose(Cmv12).reshape(Nm-NmM,NmM)
    Cmv22=Cmv[NmM:Nm,NmM:Nm]
    mv1=mv[0:NmM,0].reshape(NmM,1)
    mv2=mv[NmM:len(mv),0].reshape(Nm-NmM,1)
    m1=m[:,0:NmM]
    
    #Matriz de covarianza para la función integral
    A=Cmv22-np.matmul(CmvT12,np.matmul(Cmvinv11,Cmv12))
    #Se obtiene la inversa de la matriz de covarianza para la función integral
    Ainv=np.linalg.inv(A)
    
    #Se define un arreglo para almacenar los datos de la integral
    Integral=np.zeros((len(mn)**(NmM-1),len(mn)))
    #Se define un arreglo para almacenar
    Campana=np.zeros((len(mn)**(NmM-1),len(mn)))
    
    #Se define un vector que contiene el límite inferior de los parámetros agrupados en m2
    linf_m2=m[0,0:Nm]
    #Se define un vector que contiene el límite superior de los parámetros agrupados en m2
    lsup_m2=m[-1,0:Nm]
    
    for i in range(0,len(mn)):
        #Se obtiene el valor de la media para la marginal m1 (pp. 369, cuarta ecuación bajo la ecuación 10)
        b=mv2+np.matmul(CmvT12,np.matmul(Cmvinv11,m1[i]-mv1))
        #Se obtiene la función integral (Ec. 11, pp. 369)
        Integral[0][i]=ExpIntegral(b,Ainv,RND,V)
        #Se evalúa la función exponencial de la ec. 11
        Campana[0][i]=np.exp(-0.5*np.matmul(np.transpose(m1[i]-mv1),np.matmul(Cmvinv11,m1[i]-mv1)))
    
    #Se obtiene el producto de la función exponencial y de la función integral de la marginal (Ec. 11, pp. 369)                
    sigmMb1=Campana[0]*Integral[0]
    #Se normaliza la marginal en el intervalo de truncamiento
    sigmMb1=sigmMb1/np.trapz(sigmMb1,x=mn,dx=(mn[1]-mn[0]))
    
    #np.savetxt("marginal_m"+str(Ind[0])+"_alpha8_sob.txt", sigmMb1) #Descomentar para escribir un archivo de texto con los datos de la marginal
    
    return sigmMb1

In [7]:
#Función que calcula la media, desviación estandar y coeficiente de variación de las marginales
#Entradas
#marg: Vector que contiene los valores de la marginal
#x: Vector que contiene los valores en los que se evalúa la marginal (intervalo de truncamiento de la marginal)
#Salida
#Mean: Media de la marginal
#Sd: Desviación estandar de la marginal
#CV: Coeficiente de variación de la marginal

def Estadisticas(marg,x):
    #Se calcula la media de la marginal
    Mean=np.trapz(marg*x,x=x,dx=(x[1]-x[0]))
    #Se calcula la desviación estandar de la marginal
    Sd=(np.trapz(marg*(x-Mean)**2,x=x,dx=(x[1]-x[0])))**(0.5)
    #Se calcula el coeficiente de variación de la marginal
    CV=(Sd/Mean)*100
    
    return Mean,Sd,CV

In [8]:
#Se define el número de datos (3 componentes por 11 estaciones)
Nd=33
#Se define el número de parámetros (Discretización de falla de 18x18)
Nm=324
#Discretización de cada parámetro 
Ndm=100

In [9]:
#----------------------------Lectura y asignación de datos----------------------------

#Se extraen los datos de la matriz G
Gfile=open("G324.txt", "r")
Gstr=Gfile.read()
Gstrsplit=Gstr.split()

#Se extraen los datos de las estaciones
dfile=open("d.txt", "r")
dstr=dfile.read()
dstrsplit=dstr.split()

#Se define un arreglo para la matriz G
G=np.zeros((Nd,Nm))
#Se define un arreglo para los datos
d=np.zeros((Nd,1))

#Se almacenan los datos de la matriz G y el vector d en los arreglos correspondientes
for i in range (0,Nd):
    G[i,:]=np.array(Gstrsplit[(i*(Nm)):(i*(Nm))+Nm])
    d[i][0]=float(dstrsplit[i])
    
#Se define la transpuesta de la matriz G   
GT=np.transpose(G)

In [10]:
#----------------------------Matriz de covarianza de los datos----------------------------
#Se definen los valores de varianza de cada componente por estación
Vd=[2.1**2,2.5**2,5.1**2]*int(Nd/3)
#Se convierte la lista de valores a un arreglo numpy y las unidades a metros
Vd=np.array(Vd)*(10**-3)
#Se especifíca el vector de varianzas como vector columna
Vd=Vd.reshape(Nd,1)
#Se define una matriz cuya diagonal corresponde al vector de varianzas para obtener la matriz de covarianza de los datos
Cd=Vd*np.identity(Nd)
#Se obtiene la inversa de la matriz de covarianza de los datos
Cdinv=np.linalg.inv(Cd)

In [11]:
#------------------------Espacio de parámetros------------------------

#Se define el límite inferior de los parámetros
liminf=-0.004931
#Se define el límite superior de los parámetros
limsup=4.931506

#Se define un vector con el límite inferior de cada parámetro
a=np.zeros((1,Nm))+liminf
#Se define un vector con el límite superior de cada parámetro
b=np.zeros((1,Nm))+limsup
#Se define el valor de alpha
alpha=8
#Se define un vector con los valores posibles para un parámetro
mn=np.linspace(liminf,limsup,Ndm)
#Se define una matriz que representa el espacio de parámetros
m=np.zeros((len(mn),Nm))+mn.reshape(Ndm,1)
#Se define un vector con la media de los parámetros (pp 372, primer parrafo, tercer renglón)
m0=np.zeros((Nm,1))+(limsup/2)

In [12]:
#Se calcula el vector de medias y la matriz de covarianza de la posterior
mv_c,Cmv_c=mvCmv(G,d,Cdinv,alpha,limsup,m0)

In [13]:
#Se define la potencia de dos para el número de muestras en la secuencia de Sobol
Nom=5
#Se define un vector que contiene el límite inferior de los parámetros
linf_params=m[0,0:Nm-1]
#Se define un vector que contiene el límite superior de los parámetros
lsup_params=m[-1,0:Nm-1]
#Se define el hipervolumen de la región de integración
V_params=(liminf-limsup)**(Nm-1)
#Se define la secuencia utilizada para estimar las integrales
sampler=qmc.Sobol(d=Nm-1, scramble=True, seed=10)
sample=sampler.random_base2(m=Nom)
RND_params=qmc.scale(sample, linf_params, lsup_params)

In [14]:
#Se define un arreglo para almacenar los valores de coeficiente de variación de las marginales
CV=np.zeros(Nm)

#Se inicia un ciclo para recorrer todos los parámetros
for i in range(0,Nm):
    #Se toma un tiempo de inicio para medir el tiempo de ejecución 
    start_MCMC=time.perf_counter()
    #Se obtiene la marginal del i-ésimo parámetro
    Sig_maux=marginal_MC(m,Cmv_c,mv_c,[i+1],RND=RND_params,V=V_params)
    #Se toma el tiempo final para medir el tiempo de ejecución 
    end_MCMC=time.perf_counter()
    #Se calcula el tiempo de ejecución 
    t_MCMC=end_MCMC-start_MCMC
    print("Parámetro "+str(i+1)+" de "+str(Nm)+" tardó "+str(t_MCMC)+" segundos")
    #Se caclula la media, la desviación estandar y el coeficiente de variación del i-ésimo parámetro
    mean,sd,cv=Estadisticas(Sig_maux,mn)
    #Se almacena el valor de coeficiente de variación del i-ésimo parámetro
    CV[i]=cv

Parámetro 1 de 324 tardó 0.25645566399907693 segundos
Parámetro 2 de 324 tardó 0.10876946600183146 segundos
Parámetro 3 de 324 tardó 0.10995606299911742 segundos
Parámetro 4 de 324 tardó 0.15286369300156366 segundos
Parámetro 5 de 324 tardó 0.12823884099998395 segundos
Parámetro 6 de 324 tardó 0.1069261220000044 segundos
Parámetro 7 de 324 tardó 0.10655590000169468 segundos
Parámetro 8 de 324 tardó 0.12199707099716761 segundos
Parámetro 9 de 324 tardó 0.10714670000015758 segundos
Parámetro 10 de 324 tardó 0.12743644500005757 segundos
Parámetro 11 de 324 tardó 0.10704515100223944 segundos
Parámetro 12 de 324 tardó 0.14841813599923626 segundos
Parámetro 13 de 324 tardó 0.11781999300001189 segundos
Parámetro 14 de 324 tardó 0.11684731900095358 segundos
Parámetro 15 de 324 tardó 0.11064056999748573 segundos
Parámetro 16 de 324 tardó 0.11602370900072856 segundos
Parámetro 17 de 324 tardó 0.10658768700159271 segundos
Parámetro 18 de 324 tardó 0.12041844199848128 segundos
Parámetro 19 de 324 

Parámetro 151 de 324 tardó 0.10818520300017553 segundos
Parámetro 152 de 324 tardó 0.11776335199829191 segundos
Parámetro 153 de 324 tardó 0.10974813700158847 segundos
Parámetro 154 de 324 tardó 0.1186210209998535 segundos
Parámetro 155 de 324 tardó 0.10738064299948746 segundos
Parámetro 156 de 324 tardó 0.11947804100054782 segundos
Parámetro 157 de 324 tardó 0.10777987799883704 segundos
Parámetro 158 de 324 tardó 0.1487381849983649 segundos
Parámetro 159 de 324 tardó 0.11192586600009236 segundos
Parámetro 160 de 324 tardó 0.13600563699947088 segundos
Parámetro 161 de 324 tardó 0.11037152800054173 segundos
Parámetro 162 de 324 tardó 0.12179975099934381 segundos
Parámetro 163 de 324 tardó 0.112194866000209 segundos
Parámetro 164 de 324 tardó 0.12225869499889086 segundos
Parámetro 165 de 324 tardó 0.11248732000240125 segundos
Parámetro 166 de 324 tardó 0.11630088799938676 segundos
Parámetro 167 de 324 tardó 0.10783395299949916 segundos
Parámetro 168 de 324 tardó 0.1365188029994897 segund

Parámetro 299 de 324 tardó 0.10997225499886554 segundos
Parámetro 300 de 324 tardó 0.1493334730002971 segundos
Parámetro 301 de 324 tardó 0.11516626099910354 segundos
Parámetro 302 de 324 tardó 0.11987411299924133 segundos
Parámetro 303 de 324 tardó 0.12841349299924332 segundos
Parámetro 304 de 324 tardó 0.11510657300095772 segundos
Parámetro 305 de 324 tardó 0.11463155100136646 segundos
Parámetro 306 de 324 tardó 0.14715441800217377 segundos
Parámetro 307 de 324 tardó 0.11567282900068676 segundos
Parámetro 308 de 324 tardó 0.11915827399934642 segundos
Parámetro 309 de 324 tardó 0.14318950599772506 segundos
Parámetro 310 de 324 tardó 0.10883290800120449 segundos
Parámetro 311 de 324 tardó 0.10699939299956895 segundos
Parámetro 312 de 324 tardó 0.11615931800042745 segundos
Parámetro 313 de 324 tardó 0.12064548800117336 segundos
Parámetro 314 de 324 tardó 0.11149807299807435 segundos
Parámetro 315 de 324 tardó 0.10668051199900219 segundos
Parámetro 316 de 324 tardó 0.12837438600035966 se